In [1]:
# !pip install MulticoreTSNE
# !git clone https://github.com/elch10/anomaly-detection.git
# %cd anomaly-detection

Cloning into 'anomaly-detection'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 494 (delta 50), reused 57 (delta 29), pack-reused 409
Receiving objects: 100% (494/494), 59.66 MiB | 23.84 MiB/s, done.
Resolving deltas: 100% (310/310), done.
/content/anomaly-detection


In [2]:
import numpy as np
import pandas as pd
import keras

from src.features.build_features import rolling_window

Using TensorFlow backend.


In [ ]:
window_length = 64

tep = pd.read_csv('../tep_data.csv', index_col='Index')
prepared = rolling_window(tep, window_length)

In [6]:
M = 8

model = keras.Sequential([
  keras.layers.Conv1D(
      input_shape=(window_length, tep.shape[1]),
      filters=M,
      kernel_size=window_length // 4 + 1,
      strides=4,
      activation='relu',
  ), # output (batch, 12, M)
  keras.layers.Reshape((-1, 1, M)), # output (batch, 12, 1, M)
  keras.layers.Conv2DTranspose(
      filters=tep.shape[1],
      kernel_size=(20,1),
      strides=(4, 1),
      data_format="channels_last"
  ),
  keras.layers.Reshape((-1, tep.shape[1]))
])

model.compile('adam', loss='mse')

In [ ]:
def fit_generator(X, y, batch_size=64):
    while True:
        idxs = np.random.randint(len(X), size=batch_size)
        yield np.take(X, idxs, axis=0), np.take(y, idxs, axis=0)

def val_generator(X, y, batch_size=64):
    while True:
        for i in range(0, len(X), batch_size):
            idxs = list(range(i, min(i+batch_size, len(X))))
            yield np.take(X, idxs, axis=0), np.take(y, idxs, axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_te = train_test_split(prepared, train_size=0.7)

In [9]:
batch_size = 128
hist = model.fit_generator(
    fit_generator(X_tr, X_tr, batch_size),
    steps_per_epoch=50,
    epochs=20,
    validation_data=val_generator(X_te, X_te, batch_size),
    validation_steps=(len(X_te) + batch_size - 1) // batch_size,
    verbose=1,
    callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3),
               keras.callbacks.EarlyStopping(min_delta=0.01, patience=3, restore_best_weights=True)]
)




Epoch 1/20





50/50 [==============================] - 20s 409ms/step - loss: 0.7355 - val_loss: 0.5532
Epoch 2/20
50/50 [==============================] - 6s 119ms/step - loss: 0.4992 - val_loss: 0.4621
Epoch 3/20
50/50 [==============================] - 6s 120ms/step - loss: 0.4491 - val_loss: 0.4348
Epoch 4/20
50/50 [==============================] - 6s 121ms/step - loss: 0.4275 - val_loss: 0.4199
Epoch 5/20
50/50 [==============================] - 6s 120ms/step - loss: 0.4154 - val_loss: 0.4104
Epoch 6/20
50/50 [==============================] - 6s 119ms/step - loss: 0.4059 - val_loss: 0.4031
Epoch 7/20
50/50 [==============================] - 6s 120ms/step - loss: 0.3989 - val_loss: 0.3971
Epoch 8/20
50/50 [==============================] - 6s 121ms/step - loss: 0.3936 - val_loss: 0.3920
Epoch 9/20
50/50 [==============================] - 6s 120ms/step - loss: 0.3870 - val_loss: 0.3875
Epoch 10/20
50/50 [==============================] - 6s 118ms/step - loss: 0.3832 - val_los

In [ ]:
model.save('ts-autoencoder.h5')